# **Testing**

On utilise les poids sauvegardés après l'entrainement de notre modèle pour refaire l'inférence sur ce modèle.

# Preprocessing

## Installations

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

## Imports et Dossiers

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import json
import os
from detectron2.structures import BoxMode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Data est le dosssier mère contenant les sous dossiers test et train
Data = "drive/MyDrive/Mines Nancy/Depinfo/Projet/Faster_rCNN/Data_detectron"

Model = "drive/MyDrive/Mines Nancy/Depinfo/Projet/Faster_rCNN/Data_detectron/trained_model/model_final.pth"
Download_path = "drive/MyDrive/Mines Nancy/Depinfo/Projet/Faster_rCNN/Data_detectron/resultats"

train_path = Data + "/train"
test_path = Data + "/test"
datasets_path = {"test" : test_path, "train" : train_path}

COLORS = ['b', 'r', 'm', 'y', 'w', 'k']
category_id = {"smoke" : "0"}
print(len(os.listdir(Data)))
images = os.listdir(Data)

## Enregistrement du Dataset

In [ ]:
def get_dicts(imgdir):
    json_file = imgdir+"/dataset.json"
    with open(json_file) as f:
        dataset_dicts = json.load(f)
    for i in dataset_dicts:
        filename = i["file_name"] 
        i["file_name"] = imgdir+"/"+filename 
        for j in i["annotations"]:
            j["bbox_mode"] = BoxMode.XYWH_ABS
            label = j["category_id"]
            if label == "##::OTHER":
              label = 1
            else:
              j["category_id"] = int(label)
    return dataset_dicts

from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "test"]:
    DatasetCatalog.register("data_dict_" + d, lambda d=d: get_dicts(datasets_path[d]))
    MetadataCatalog.get("data_dict_" + d).set(thing_classes=["smoke"])
metadata_dict = MetadataCatalog.get("data_dict_train")

In [ ]:
# Visualiser le dataset de test
dataset_dicts = get_dicts(test_path)

for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    print(d)
    visualizer = Visualizer(img[:, :, ::-1], metadata=.)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

## Configuration

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Permet de fixer le score minimal d'acceptation d'une image
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.MODEL.WEIGHTS = Model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
predictor = DefaultPredictor(cfg)

# Predictions

In [ ]:
# Prediction sur le dataset de test
from detectron2.utils.visualizer import ColorMode
dataset_test = get_dicts(test_path)
for i in range(10): 
    d = dataset_test[i]   
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    print(d)

    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata_dict, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
# Autre moyen d'avoir la detection, sans avoir à enregistrer les metadonnées sur on veut faire un test sur une image isolée
from detectron2.utils.visualizer import ColorMode
metadata = MetadataCatalog.get("__nonexist__")

im = cv2.imread(os.path.join(Data, images[0]))
outputs = predictor(im)
print(images[0])
v = Visualizer(im[:, :, ::-1], metadata = metadata, scale=0.8, instance_mode=ColorMode.IMAGE )
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
prediction = v.get_image()[:,:,::-1]
cv2_imshow(prediction)
print(type(prediction))
cv2.imwrite(os.path.join(Download_path, images[0]), v.get_image()[:, :, ::-1])

# Evaluation du modèle sur la base de test

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_dict_test", cfg, False, output_dir="/output/")
val_loader = build_detection_test_loader(cfg, "data_dict_test")
inference_on_dataset(predictor.model, val_loader, evaluator)